# Homework 9.1: Analysis of microtubule catastrophe (55 pts)

[Dataset download](https://s3.amazonaws.com/bebi103.caltech.edu/data/gardner_mt_catastrophe_only_tubulin.csv)

<hr>

**Be sure to read problem 9.3 before beginning this problem.**

Refresh yourself about the microtubule catastrophe data we explored in previous homeworks. The data set we used there was from a control experiment wherein [Garner, Zanic, and coworkers](https://doi.org/10.1016/j.cell.2011.10.037) compared measurements of microtubule catastrophe times for experiments using fluorescently labeled tubulin and unlabeled tubulin. In this problem you will investigate microtubule catastrophe using labeled tubulin for different concentrations of total tubulin. In the analysis that follows, you will very likely use code you have written in previous homeworks. This also underscores the importance of writing good, well-documented, well-tested code, since you will almost always end up using it again.

You can download the data set for  microtubule catastrophe times for various concentrations of tubulin here: [https://s3.amazonaws.com/bebi103.caltech.edu/data/gardner_mt_catastrophe_only_tubulin.csv](https://s3.amazonaws.com/bebi103.caltech.edu/data/gardner_mt_catastrophe_only_tubulin.csv).

**a)** I shouldn't have to say this, but perform an exploratory data analysis on the data set.

**b)** In homework 7.2, you performed parameter estimates for two models of microtubule catastrophe. In one model, the catastrophe times were Gamma distributed, and in another, they were distributed according to the following story: The arrival of two successive Poisson processes (with different rates) immediately trigger microtubule catastrophe. Use the data set with 12 µM tubulin to compare these two models. Which model is perferred?

**c)** Using whichever model you favor based on your work in part (b), obtain parameter estimates for the other tubulin concentrations. Given that microtubules polymerize faster with higher tubulin concentrations, is there anything you can say about the occurrence of catastrophe by looking at the values of the parameters versus tubulin concentration?

In [49]:
import numpy as np
import pandas as pd
import warnings
import bokeh.io
import bokeh.plotting
from bokeh.models import Label
bokeh.io.output_notebook()
from bokeh.plotting import figure, output_file, show
import iqplot
import scipy.special

import random

%load_ext blackcellmagic

import scipy.optimize
import scipy.stats as st

import bebi103
import numba
import iqplot
from bokeh.layouts import row
import bokeh.io
bokeh.io.output_notebook()

# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade iqplot bebi103 watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    data_path = "https://s3.amazonaws.com/bebi103.caltech.edu/data/"
else:
    data_path = "../data/"
# ------------------------------

Loading BokehJS ...

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


Loading BokehJS ...

Loading and tidying the dataset

In [3]:
# Load in DataFrames
data_path = "../data/"
df = pd.read_csv(
    os.path.join(data_path, "gardner_mt_catastrophe_only_tubulin.csv"), index_col=0, comment="#"
)


df1 = df.reset_index()



df1.columns.names = ['concentrations']


df1['row_num'] = range(1, len(df) + 1)



df2 = pd.melt(df1, id_vars = ['row_num'], var_name = 'concentration', value_name = 'catastrophe_time')


df2.drop('row_num', axis=1, inplace=True)
df3 = df2.dropna()




EDA plots 

In [4]:


p = iqplot.ecdf(
    data=df3, q="catastrophe_time", conf_int=True
)
p.title = "Time to microtubule Catastrophe "




df3['concentration (µM)'] = df3['concentration'].apply(lambda x: float(x[:x.rfind(' ')]))

p1 = bokeh.plotting.figure(
    frame_height=300,
    frame_width=400,
    x_axis_label="catastrophe_time",
    y_axis_label="concentration",

)

p1.circle(
    source=df3,
    x="catastrophe_time",
    y="concentration (µM)",
    alpha=1,
)

bokeh.io.show(p)

bokeh.io.show(p1)

df3.tail()



/var/folders/bx/f26vfc2x3lx1dmd831hkpdkw0000gn/T/ipykernel_42829/617268960.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['concentration (µM)'] = df3['concentration'].apply(lambda x: float(x[:x.rfind(' ')]))


,concentration,catastrophe_time,concentration (µM)
2904,14 uM,1005.0,14.0
2905,14 uM,1135.0,14.0
2906,14 uM,1305.0,14.0
2907,14 uM,1400.0,14.0
2908,14 uM,1420.0,14.0


In [51]:
concs=["7 uM","9 uM","10 uM","12 uM","14 uM"]

c7 = df3.loc[df3["concentration"] == "7 uM", "catastrophe_time"].values
c9 = df3.loc[df3["concentration"] == "9 uM", "catastrophe_time"].values
c10 = df3.loc[df3["concentration"] == "10 uM", "catastrophe_time"].values
c12 = df3.loc[df3["concentration"] == "12 uM", "catastrophe_time"].values
c14 = df3.loc[df3["concentration"] == "14 uM", "catastrophe_time"].values


p = iqplot.ecdf(
        data=c7, q="catastrophe_time", title ="7 uM concentration", conf_int=True
)
q = iqplot.ecdf(
        data=c9, q="catastrophe_time", title ="9 uM concentration",conf_int=True
)

r = iqplot.ecdf(
        data=c10, q="catastrophe_time", title ="10 uM concentration", conf_int=True
)


s = iqplot.ecdf(
        data=c12, q="catastrophe_time", title = "12 uM concentration", conf_int=True
)

t = iqplot.ecdf(
        data=c14, q="catastrophe_time", title ="14 uM concentration", conf_int=True
)

bokeh.io.show(row(p,q,r,s,t))



Selecting data for the 12 uM concentration

In [8]:


conc12 = df3.loc[df3["concentration"] == "12 uM", "catastrophe_time"].values



Functions to get the log likelihood and MLEs from a gamma distribution

In [21]:
def log_like_gamma(params, n):
    """Log likelihood for i.i.d. Gamma measurements, parametrized
    by alpha, b=1/b."""
    alpha, b = params

    if alpha <= 0 or b <= 0:
        return -np.inf

    return np.sum(st.gamma.logpdf(n, alpha, loc=0, scale=1/b))

def gamma_mle(n):
    """Perform maximum likelihood estimates for parameters for i.i.d.
    Gamma measurements, parametrized by alpha, b=1/b"""
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        res = scipy.optimize.minimize(
            fun=lambda params, n: -log_like_gamma(params, n),
            x0=np.array([3, 3]),
            args=(n,),
            method='Powell'
        )

    if res.success:
        return res.x
    else:
        raise RuntimeError('Convergence failed with message', res.message)
        

        
alpha_mle, beta_mle = gamma_mle(conc12)

alpha_mle, beta_mle

(2.914939872465886, 0.007661228056591428)

Plotting ecdf for data drawn out of the theoretical distribution

In [11]:

p = iqplot.ecdf(conc12, q= "time", conf_int=True)

t_theor = np.linspace(0, 2000, 200)
cdf = st.gamma.cdf(t_theor, alpha_mle, loc=0, scale=1/beta_mle)
p.line(t_theor, cdf, line_width=2, color='orange')

bokeh.io.show(p)


Getting confidence intervals for the data using MLE of the gamma distribution

In [12]:


def gen_gamma(params, size, rg):
    alpha, beta = params
    return rg.gamma(alpha, 1 / beta, size=size)

bs_reps = bebi103.bootstrap.draw_bs_reps_mle(
    mle_fun=gamma_mle, 
    gen_fun=gen_gamma, 
    data=conc12, 
    size=500, 
    n_jobs=1,
    progress_bar=True,
)

CI = np.percentile(bs_reps, [2.5, 97.5], axis=0)
CI

100%|█████████████████████████████████████████| 500/500 [00:29<00:00, 16.69it/s]


array([[2.63951493, 0.0069083 ],
       [3.22893708, 0.00863175]])

**Model 2, 2 step poisson process, we used code from 7.2 functions to get the MLEs for this second custom model** 

In [13]:

def log_like(beta1, t):
    """Compute the log likelihood for a given value of β1,
    assuming Δβ is set so that the dervitive of the log
    likelihood with respect to β1 vanishes."""
    n = len(t)
    tbar = np.mean(t)
    beta1_tbar = beta1 * tbar
    
    if beta1_tbar > 2 or beta1_tbar < 1:
        return np.nan

    if np.isclose(beta1_tbar, 2):
        return -2 * n * (1 + np.log(tbar) - np.log(2)) + np.sum(np.log(t))
        
    if np.isclose(beta1_tbar, 1):
        return -n * (1 + np.log(tbar))
            
    delta_beta = beta1 * (2 - beta1 * tbar) / (beta1 * tbar - 1)

    ell = n * (np.log(beta1) + np.log(beta1 + delta_beta) - np.log(delta_beta))
    ell -= n * beta1_tbar
    ell += np.sum(np.log(1 - np.exp(-delta_beta * t)))
    
    return ell

def dlog_like_dbeta1(beta1, t):
    """Returns the derivative of the log likelihood w.r.t. Δβ
    as a function of β1, assuming Δβ is set so that the dervitive 
    of the log likelihood with respect to β1 vanishes."""
    n = len(t)
    tbar = np.mean(t)
    beta1_tbar = beta1 * tbar
    
    if beta1_tbar > 2 or beta1_tbar < 1:
        return np.nan

    if np.isclose(beta1_tbar, 2) or np.isclose(beta1_tbar, 1):
        return 0.0
            
    delta_beta = beta1 * (2 - beta1 * tbar) / (beta1 * tbar - 1)
    
    exp_val = np.exp(-delta_beta * t)
    sum_term = np.sum(t * exp_val / (1 - exp_val))
    
    return -n / delta_beta + n / (beta1 + delta_beta) + sum_term

def mle_two_step(t, nbeta1=500):
    """Compute the MLE for the two-step model."""
    # Compute ∂ℓ/∂Δβ for values of beta_1
    tbar = np.mean(t)
    beta1 = np.linspace(1 / tbar, 2 / tbar, nbeta1)
    deriv = np.array([dlog_like_dbeta1(b1, t) for b1 in beta1])
    
    # Add the roots at the edges of the domain
    beta1_vals = [1 / tbar, 2 / tbar]
    ell_vals = [log_like(beta1_vals[0], t), log_like(beta1_vals[1], t)]
    
    # Find all sign flips between the edges of the domain
    sign = np.sign(deriv[1:-1])
    inds = np.where(np.diff(sign))[0]
    
    # Perform root finding at the sign flips
    for i in inds:
        b1 = scipy.optimize.brentq(dlog_like_dbeta1, beta1[i+1], beta1[i+2], args=(t,))
        beta1_vals.append(b1)
        ell_vals.append(log_like(b1, t))
        
    # Find the value of beta1 that gives the maximal log likelihood
    i = np.argmax(ell_vals)
    beta1 = beta1_vals[i]

    # Compute beta 2
    if np.isclose(beta1, 1 / tbar):
        delta_beta = np.inf
    else:
        delta_beta = beta1 * (2 - beta1 * tbar) / (beta1 * tbar - 1)

    beta2 = beta1 + delta_beta
    
    return np.array([beta1, beta2])


beta1, beta2 = mle_two_step(conc12)

beta1, beta2

(0.005255498728830042, 0.005255498728830042)

Getting confidence intervals for model 2

In [14]:
rg = np.random.default_rng(seed=3252)


def draw_model2(betas, size, rg):
    beta1, beta2 = betas
    return rg.exponential(1 / beta1, size=size) + rg.exponential(1 / beta2, size=size)


bs_reps_two_step = bebi103.bootstrap.draw_bs_reps_mle(
    mle_two_step, draw_model2, conc12, size=500, n_jobs=1
) #ran 500 bootstraps to save time 
CI = np.percentile(bs_reps_two_step, [2.5, 97.5], axis=0)
CI

/Users/tm/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4527: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


array([[0.0025712 , 0.00505325],
       [0.00548325,        nan]])

Model 1 seems to have tigher confidence intervals but they do not tell us much about the which model is better, we employ the predictive plots and akaike methods to assess which model is closer to the generative distribution of these data. 

**Model 1**

In [16]:


alpha_mle, beta_mle = gamma_mle(conc12)

alpha_mle, beta_mle



rg = np.random.default_rng(seed=3252)

gamma_samples = np.array(
    [rg.gamma(alpha_mle, 1 / beta_mle, size=len(conc12)) for _ in range(100000)]
)

gamma_samples


#graphical model assessment (predictive graphs) 

@numba.njit
def ecdf(x, data):
    """Give the value of an ECDF at arbitrary points x."""
    y = np.arange(len(data) + 1) / len(data)
    return y[np.searchsorted(np.sort(data), x, side="right")]

n_theor = np.arange(0, gamma_samples.max() + 1)

ecdfs = np.array([ecdf(n_theor, sample) for sample in gamma_samples])

ecdf_low, ecdf_high = np.percentile(ecdfs, [2.5, 97.5], axis=0)


#plot

p = bebi103.viz.fill_between(
    x1=n_theor,
    y1=ecdf_high,
    x2=n_theor,
    y2=ecdf_low,
    patch_kwargs={"fill_alpha": 0.5},
    x_axis_label="time",
    y_axis_label="ECDF",
)

#now overlay the data 

p = iqplot.ecdf(conc12, q= "time",palette=["orange"], p=p)




#comparison between predictive ecdf and data ecdf

p1 = bebi103.viz.predictive_ecdf(
    samples=gamma_samples, data=conc12, x_axis_label="times"
)


p1 = bebi103.viz.predictive_ecdf(
    samples=gamma_samples, data=conc12, diff="ecdf", x_axis_label="times"
)


bokeh.io.show(p)
bokeh.io.show(p1)




**Model 2**

In [19]:

beta1, beta2 = mle_two_step(conc12)

beta1, beta2

#draw data out of model2

rg = np.random.default_rng(seed=3252)

def draw_model2samples(beta1 ,beta2, size, rg):
    
    return rg.exponential(1 / beta1, size=size) + rg.exponential(1 / beta2, size=size)


model2_samples = np.array(
    [draw_model2samples(beta1, beta2, size=len(conc12), rg=rg) for _ in range(100000)]
)

model2_samples



@numba.njit
def ecdf(x, data):
    """Give the value of an ECDF at arbitrary points x."""
    y = np.arange(len(data) + 1) / len(data)
    return y[np.searchsorted(np.sort(data), x, side="right")]

n_theor2 = np.arange(0, model2_samples.max() + 1)

ecdfs2 = np.array([ecdf(n_theor2, sample) for sample in model2_samples])

ecdf_low2, ecdf_high2 = np.percentile(ecdfs2, [2.5, 97.5], axis=0)

#plot with data overlayed 

p = bebi103.viz.fill_between(
    x1=n_theor2,
    y1=ecdf_high2,
    x2=n_theor2,
    y2=ecdf_low2,
    patch_kwargs={"fill_alpha": 0.5},
    x_axis_label="times",
    y_axis_label="ECDF",
)

p = iqplot.ecdf(conc12, q= "time",palette=["orange"], p=p)



p1 = bebi103.viz.predictive_ecdf(
    samples=model2_samples, data=conc12, x_axis_label="times"
)


p1 = bebi103.viz.predictive_ecdf(
    samples=model2_samples, data=conc12, diff="ecdf", x_axis_label="times"
)

bokeh.io.show(p)

bokeh.io.show(p1)


Akaike model assessment: model with the lesser AIC, or AIC weight closer to 1 is closer to the true generative distribution

**model 2** 

In [25]:
beta1, beta2 = mle_two_step(conc12)

#get loglikelihood of mles

log_like_mle2=log_like(beta1, conc12)

log_like_mle2

aic2 = -2 * log_like_mle2 - 4

aic2

#aic weights 

aic_max = max([aic1, aic2])
numerator = np.exp(-(aic2 - aic_max)/2)
denominator = numerator + np.exp(-(aic1- aic_max)/2)
aic_weight = numerator / denominator
aic2, aic_weight



(9319.370000055043, 2.2662412631198434e-11)

**Model 1**

In [26]:

alpha_mle, beta_mle = gamma_mle(conc12)
params = [alpha_mle, beta_mle]

#get loglikelihood of mles
#log_like_gamma(params, n)
log_like_mle1=log_like_gamma(params, conc12)

log_like_mle1

aic1 = -2 * log_like_mle1 - 4

aic1
    
#aic weights 

aic_max = max([aic1, aic2])
numerator1 = np.exp(-(aic1 - aic_max)/2)
denominator1 = numerator1 + np.exp(-(aic2- aic_max)/2)
aic_weight1 = numerator1 / denominator1
aic1, aic_weight1



(9270.34937326495, 0.9999999999773376)

Show AKAIKE weights for both models together and compare  

In [27]:
def Akaike(l_1, l_2):
    """
    input: the 3 MLEs for each experiment and the data for each experiment ([[time][area]])
    returns Akaike weight for the exponential and linear model. in this order.
    """
    AIC1 = -2 * l_1 + 4
    AIC2 = -2 * l_2 + 4
    w_1 = 1 / (1 + np.exp(-0.5 * (AIC2 - AIC1)))
    w_2 = 1 / (1 + np.exp(-0.5 * (AIC1 - AIC2)))
    return AIC1, w_1, AIC2,  w_2


Akaike(log_like_mle1, log_like_mle2)

(9278.34937326495,
 0.9999999999773377,
 9327.370000055043,
 2.2662412631198434e-11)

**Both graphical and AKAIKE model assessments shows preference for model one for modeling the time to catastrophe data at 12 uM concentration**

We therefore apply model 1 for a gamma distribution to get MLEs for the other 4 concentrations in our datasets


In [29]:
concs=["7 uM","9 uM","10 uM","12 uM","14 uM"]

def mle_get(df, gamma_mle, concs):
    mles = []
    for i in concs:
        vals = df3.loc[df3["concentration"] == i, "catastrophe_time"].values
        answer = [gamma_mle(vals), i ]
        mles.append(answer)
    return mles


mle_get(df3, gamma_mle, concs)

[[array([2.44385762, 0.00755037]), '7 uM'],
 [array([2.67971607, 0.00877937]), '9 uM'],
 [array([3.21079724, 0.00902989]), '10 uM'],
 [array([2.91493987, 0.00766123]), '12 uM'],
 [array([3.36146269, 0.00717499]), '14 uM']]

Using whichever model you favor based on your work in part (b), obtain parameter estimates for the other tubulin concentrations. Given that microtubules polymerize faster with higher tubulin concentrations, is there anything you can say about the occurrence of catastrophe by looking at the values of the parameters versus tubulin concentration?

**Conclusion**:From EDA we saw that higher concentrations of tubulin tended to have higher times to catastrophe, From the MLE values, higher concentrations of tubulin corresponded to higher maximum likelihood estimates of alpha and lower maximum likelihood estimates of beta. In the gamma distribution, alpha is the number of events and beta is the rate of arrivals. The higher values of alpha show that when tubulin concentration is higher, the number if steps/events/poisson processes required to trigger catastrophe is higher whereas the lower values of beta for high tubulin concentration show that rate of arrival for these events in a gamma distribution is actually slower at high concentrations.     

In [47]:
%load_ext watermark
%watermark -v -p numpy,pandas,scipy,bokeh,iqplot,bebi103,jupyterlab

Python implementation: CPython
Python version       : 3.9.13
IPython version      : 8.4.0

numpy     : 1.23.3
pandas    : 1.5.0
scipy     : 1.9.2
bokeh     : 2.4.3
iqplot    : 0.3.2
bebi103   : 0.1.13
jupyterlab: 3.4.4



Kimberley worked on 9.1